<a href="https://colab.research.google.com/github/Nikhileswar-Komati/100D_ML/blob/master/HR_FIND_BUDDY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import files
files.upload()
!unzip '808177_1384835_bundle_archive'

Saving 808177_1384835_bundle_archive.zip to 808177_1384835_bundle_archive.zip
Archive:  808177_1384835_bundle_archive.zip
  inflating: test.csv                
  inflating: train.csv               


In [3]:
train_df = pd.read_csv('/content/train.csv', index_col = 'pet_id')
test_df = pd.read_csv('/content/test.csv', index_col = 'pet_id')

In [4]:
val = train_df['length(m)'].mean()
train_df['length(m)'] = train_df['length(m)'].replace(to_replace=0, value=val)
test_df['length(m)'] = test_df['length(m)'].replace(to_replace=0, value=val)


In [5]:
from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder(handle_unknown='ignore', sparse = False)

train_X_encoded = pd.DataFrame(enc.fit_transform(train_df[['color_type']]), index = train_df.index.values)

train_X_encoded.columns = enc.get_feature_names(['color_type'])


test_X_encoded = pd.DataFrame(enc.transform(test_df[['color_type']]), index = test_df.index.values)

test_X_encoded.columns = enc.get_feature_names(['color_type'])

train_df = pd.concat([train_df, train_X_encoded ], axis=1)
test_df = pd.concat([test_df, test_X_encoded ], axis=1)

train_df['height(cm)'] = train_df['height(cm)'].apply(lambda x: x/100)
test_df['height(cm)'] = test_df['height(cm)'].apply(lambda x: x/100)

train_df.condition.fillna(3.0, inplace = True)
test_df.condition.fillna(3.0, inplace = True)

train_df['issue_date'] = pd.to_datetime(train_df['issue_date'])
train_df['listing_date'] = pd.to_datetime(train_df['listing_date'])
test_df['issue_date'] = pd.to_datetime(test_df['issue_date'])
test_df['listing_date'] = pd.to_datetime(test_df['listing_date'])

train_df['is_month'] = train_df.issue_date.dt.month
train_df['lis_month'] = train_df.listing_date.dt.month
test_df['is_month'] = test_df.issue_date.dt.month
test_df['lis_month'] = test_df.listing_date.dt.month


train_df['diff_weeks'] = train_df.listing_date - train_df.issue_date
train_df['diff_weeks'] = train_df['diff_weeks'] / np.timedelta64(1, 'W')

train_df['X1_and_X2'] = train_df.X1 * train_df.X2
train_df['X1_plus_X2'] = train_df.X1 + train_df.X2

train_df['length_and_height'] = train_df['height(cm)'] * train_df['length(m)']
train_df['length_plus_height'] = train_df['height(cm)'] + train_df['length(m)']

train_df.drop(train_df[train_df.diff_weeks < 0].index, inplace=True)

test_df['diff_weeks'] = test_df.listing_date - test_df.issue_date
test_df['diff_weeks'] = test_df['diff_weeks'] / np.timedelta64(1, 'W')

test_df['X1_and_X2'] = test_df.X1 * test_df.X2
test_df['X1_plus_X2'] = test_df.X1 + test_df.X2

test_df['length_and_height'] = test_df['height(cm)'] * test_df['length(m)']
test_df['length_plus_height'] = test_df['height(cm)'] + test_df['length(m)']

test_df.drop(test_df[test_df.diff_weeks < 0].index, inplace=True)


In [6]:
test_df.sample()

,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,color_type_Agouti,color_type_Apricot,color_type_Black,color_type_Black Brindle,color_type_Black Smoke,color_type_Black Tabby,color_type_Black Tiger,color_type_Blue,color_type_Blue Cream,color_type_Blue Merle,color_type_Blue Point,color_type_Blue Smoke,color_type_Blue Tabby,color_type_Blue Tick,color_type_Blue Tiger,color_type_Brown,color_type_Brown Brindle,color_type_Brown Merle,color_type_Brown Tabby,color_type_Brown Tiger,color_type_Buff,color_type_Calico,color_type_Calico Point,color_type_Chocolate,color_type_Chocolate Point,color_type_Cream,color_type_Cream Tabby,color_type_Fawn,color_type_Flame Point,color_type_Gold,color_type_Gray,color_type_Gray Tabby,color_type_Green,color_type_Lilac Point,color_type_Liver,color_type_Liver Tick,color_type_Lynx Point,color_type_Orange,color_type_Orange Tabby,color_type_Pink,color_type_Red,color_type_Red Merle,color_type_Red Tick,color_type_Sable,color_type_Seal Point,color_type_Silver,color_type_Silver Lynx Point,color_type_Silver Tabby,color_type_Tan,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,is_month,lis_month,diff_weeks,X1_and_X2,X1_plus_X2,length_and_height,length_plus_height
pet_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ANSL_54252,2015-07-28,2016-08-22 17:26:00,1.0,Cream Tabby,0.42,0.1575,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,8,55.960913,0,1,0.06615,0.5775


In [7]:
Y_breed_type = train_df['breed_category']
Y_pet_type = train_df['pet_category']

train_df = train_df.drop(['breed_category', 'pet_category', 'issue_date', 'listing_date', 'color_type', 'X1', 'X2'], axis=1)
test_df = test_df.drop(['issue_date', 'listing_date', 'color_type', 'X1', 'X2'], axis=1)

train_X_breed = train_df.values
train_Y_breed = Y_breed_type.values
train_Y_pet = Y_pet_type.values

test_X_breed = test_df.values

In [8]:
from xgboost import XGBClassifier

model1 = XGBClassifier()

model1.fit(train_X_breed, train_Y_breed)

#Predict Output
predicted_Y_breed = model1.predict(test_X_breed)
predicted_Y_breed_arr = predicted_Y_breed


In [9]:
predicted_X_breed = model1.predict(train_X_breed)
predicted_X_breed = np.reshape(predicted_X_breed, (predicted_X_breed.shape[0], 1))
predicted_Y_breed_arr = np.reshape(predicted_Y_breed_arr, (predicted_Y_breed_arr.shape[0], 1))
train_X_pet = np.append(train_X_breed, predicted_X_breed, axis = 1)
test_X_pet = np.append(test_X_breed, predicted_Y_breed_arr, axis = 1)

In [10]:
train_X_pet.shape

(18832, 67)

In [11]:
from xgboost import XGBClassifier

model1 = XGBClassifier()

model1.fit(train_X_pet, train_Y_pet)

#Predict Output
predicted_Y_pet = model1.predict(test_X_pet)


In [12]:
final = test_df.copy()

final['breed_category'] = predicted_Y_breed
final['pet_category'] = predicted_Y_pet

cols = test_df.columns
final = final.drop(cols, axis=1)

final.to_csv('submit_9.csv')

In [ ]:
final.to_csv('submit_7.csv')